<a href="https://colab.research.google.com/github/Diksha-Bisht/Document-Q-A-with-RAG-and-LangChain/blob/main/Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Load Document

In [8]:
!pip install python-docx PyMuPDF --quiet

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.3/244.3 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 62.3 MB/s eta 0:00:00


In [9]:
from google.colab import files
import fitz  # PyMuPDF
from docx import Document as DocxDocument

# Upload files
uploaded_files = files.upload()

# Store loaded documents
documents = []

# Function to extract text based on file type
def extract_text_from_file(filename):
    text = ""
    if filename.endswith(".pdf"):
        doc = fitz.open(filename)
        for page in doc:
            text += page.get_text()
    elif filename.endswith(".txt"):
        with open(filename, "r", encoding="utf-8") as f:
            text = f.read()
    elif filename.endswith(".docx") or filename.endswith(".doc"):
        doc = DocxDocument(filename)
        for para in doc.paragraphs:
            text += para.text + "\n"
    else:
        print(f"Unsupported file type: {filename}")
    return text

# Read all uploaded files
for filename in uploaded_files.keys():
    content = extract_text_from_file(filename)
    documents.append({"filename": filename, "content": content})

# Print preview
for doc in documents:
    print(f"\n📄 File: {doc['filename']}\nContent Preview:\n{doc['content'][:500]}...\n{'='*80}")


Saving Diksha_Resume.pdf to Diksha_Resume.pdf

📄 File: Diksha_Resume.pdf
Content Preview:
Diksha Bisht
bishtdiksha096@gmail.com | +91 9548260152 | myportfolio.com | diksha-bisht-781314213 | Diksha-Bisht
Skills
Programming: Python, SQL
Tools: Power BI, Tableau, Looker Studio, Git, Jupyter, Excel & Microsoft Tools, Streamlit, Fast API, AWS
Libraries: Scikit-learn, NumPy, Pandas, Matplotlib, TensorFlow, OpenCV
Concepts: Data Preprocessing, Model Tuning, EDA, Supervised & Unsupervised Learning, Model Evaluation, NLP,
Cloud & Deployment
Projects
Document Q&A with RAG and LangChain
• Built...


In [13]:
type(documents)

list

In [15]:
len(documents)

1

#Split Document into Chunks

In [32]:
len(documents[0]['content']) / 1000     # Number of chunks created

2.34

In [16]:
!pip install langchain --quiet

In [17]:
from langchain.text_splitter import CharacterTextSplitter

# Define the text splitter
text_splitter = CharacterTextSplitter(
    separator="\n",
    chunk_size=1000,      # You can adjust based on LLM token limit
    chunk_overlap=200,    # Prevent context loss between chunks
    length_function=len
)

# Prepare the final list of chunks
all_chunks = []

# Split content of each uploaded document
for doc in documents:
    chunks = text_splitter.split_text(doc["content"])
    for i, chunk in enumerate(chunks):
        all_chunks.append({
            "source": doc["filename"],
            "chunk_id": i,
            "content": chunk
        })

# Preview a few chunks
for chunk in all_chunks[:3]:
    print(f"\n📦 Chunk {chunk['chunk_id']} from {chunk['source']}:\n{chunk['content'][:300]}...\n{'-'*80}")



📦 Chunk 0 from Diksha_Resume.pdf:
Diksha Bisht
bishtdiksha096@gmail.com | +91 9548260152 | myportfolio.com | diksha-bisht-781314213 | Diksha-Bisht
Skills
Programming: Python, SQL
Tools: Power BI, Tableau, Looker Studio, Git, Jupyter, Excel & Microsoft Tools, Streamlit, Fast API, AWS
Libraries: Scikit-learn, NumPy, Pandas, Matplotlib...
--------------------------------------------------------------------------------

📦 Chunk 1 from Diksha_Resume.pdf:
• Tools Used: Transformers, Datasets, Hugging Face Trainer, Streamlit
Product Usage Prediction (Netflix-style Forecasting)
• Built a machine learning model to predict user engagement with a streaming platform based on historical
behavior data.
• Tools Used: Scikit-learn, XGBoost, SQL, Matplotlib
Exp...
--------------------------------------------------------------------------------

📦 Chunk 2 from Diksha_Resume.pdf:
campaign performance, user engagement, and sales KPIs.
• Designed and implemented CRM automation workflows, improving lead tr

In [21]:
len(all_chunks)

3

# Generating Embedding

In [18]:
!pip install sentence-transformers --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 40.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 46.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 28.1 MB/s eta 0:00:00


In [19]:
from sentence_transformers import SentenceTransformer

# Load a lightweight embedding model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Fast, ~384-dimension output

# Extract text content from each chunk
chunk_texts = [chunk['content'] for chunk in all_chunks]

# Generate embeddings
embeddings = model.encode(chunk_texts, show_progress_bar=True)

# Store with metadata
embedded_chunks = []
for i, chunk in enumerate(all_chunks):
    embedded_chunks.append({
        "source": chunk["source"],
        "chunk_id": chunk["chunk_id"],
        "content": chunk["content"],
        "embedding": embeddings[i]
    })

# Preview
print(f"\n✅ Generated {len(embedded_chunks)} embeddings.")
print(f"Sample embedding (first 5 dims): {embedded_chunks[0]['embedding'][:5]}")


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]


✅ Generated 3 embeddings.
Sample embedding (first 5 dims): [-0.1235704  -0.01015104  0.00816054  0.01542321 -0.01459593]


In [33]:
embedded_chunks     #Contain Source, chunk_id, content, embedding

[{'source': 'Diksha_Resume.pdf',
  'chunk_id': 0,
  'content': 'Diksha Bisht\nbishtdiksha096@gmail.com | +91 9548260152 | myportfolio.com | diksha-bisht-781314213 | Diksha-Bisht\nSkills\nProgramming: Python, SQL\nTools: Power BI, Tableau, Looker Studio, Git, Jupyter, Excel & Microsoft Tools, Streamlit, Fast API, AWS\nLibraries: Scikit-learn, NumPy, Pandas, Matplotlib, TensorFlow, OpenCV\nConcepts: Data Preprocessing, Model Tuning, EDA, Supervised & Unsupervised Learning, Model Evaluation, NLP,\nCloud & Deployment\nProjects\nDocument Q&A with RAG and LangChain\n• Built a Retrieval-Augmented Generation (RAG) pipeline using LangChain and Hugging Face to enable intelligent\nQ&A over custom document datasets.\n• Tools Used: Pinecone, LLM, LangChain, Streamlit, Hugging Face\nCustomer Support Chatbot (Fine-Tuned LLM)\n• Developed a domain-specific customer support chatbot by fine-tuning LLaMA-2 on retail-specific queries.\n• Tools Used: Transformers, Datasets, Hugging Face Trainer, Streamlit\

In [35]:
len(embedded_chunks[0]['embedding'])

384

In [36]:
len(embedded_chunks[1]['embedding'])

384

#Store in Vector DB

In [42]:
# Uninstall the old Pinecone client
!pip uninstall -y pinecone-client

# Install the new official Pinecone SDK
!pip install -U pinecone



Found existing installation: pinecone-client 6.0.0
Uninstalling pinecone-client-6.0.0:
  Successfully uninstalled pinecone-client-6.0.0
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 421.9/421.9 kB 9.1 MB/s eta 0:00:00


In [38]:
from google.colab import userdata

In [44]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=userdata.get('PINECONE'))

index_name = "q-a-index"

if not pc.list_indexes().get(index_name):
    pc.create_index(
        name=index_name,
        dimension=384,  # must match the embedding size (384 for all-MiniLM-L6-v2)
        metric="cosine",  # cosine is standard for sentence embeddings
        spec=ServerlessSpec(cloud="aws", region="us-east-1")
    )

# Connect to the index
index = pc.Index(index_name)

In [45]:
vectors = [
    {
        "id": f"{chunk['source']}_chunk_{chunk['chunk_id']}",
        "values": chunk['embedding'].tolist(),
        "metadata": {
            "source": chunk['source'],
            "content": chunk['content']
        }
    }
    for chunk in embedded_chunks
]

index.upsert(vectors=vectors)


{'upserted_count': 3}

#Take user Query

In [46]:
# Accept user query
user_query = input("🔍 Enter your question: ")

# Embed the query using the same model
query_embedding = model.encode(user_query)

print(f"\n✅ Query: {user_query}")
print(f"🔢 Query embedding (first 5 values): {query_embedding[:5]}")


🔍 Enter your question: What are skills in this resume

✅ Query: What are skills in this resume
🔢 Query embedding (first 5 values): [-0.07085861  0.04721449 -0.01929596  0.0294111   0.02113301]


#Step 7: Retrieve Relevant Chunks from Pinecone and Pass to LLM (LangChain-compatible)

We’ll:

Query Pinecone using the embedded user query.

Extract the top matching chunks.

Feed them with the question into an LLM (e.g., OpenAI
GPT or Hugging Face model via LangChain).

In [51]:
!pip install langchain langchain-community transformers accelerate sentence-transformers pinecone-client --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 25.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 3.1 MB/s eta 0:00:00


In [52]:
from langchain_community.llms import HuggingFacePipeline
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM, pipeline

# Load FLAN-T5 base
model_name = "google/flan-t5-base"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(model_name)

# Setup Hugging Face pipeline
qa_pipeline = pipeline(
    "text2text-generation",
    model=model,
    tokenizer=tokenizer,
    max_length=512,
    temperature=0.3,
    do_sample=True,
    top_p=0.95,
    repetition_penalty=1.2
)

# Wrap in LangChain
llm = HuggingFacePipeline(pipeline=qa_pipeline)


tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

Device set to use cpu
<ipython-input-52-d30e2486494b>:22: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=qa_pipeline)


In [54]:
!pip install -U langchain langchain-community langchain-huggingface langchain-pinecone transformers sentence-transformers pinecone-client --quiet


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 345.7/345.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 43.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 437.6/437.6 kB 24.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.2/52.2 kB 3.1 MB/s eta 0:00:00


In [57]:
# LangChain imports from correct packages
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_pinecone import PineconeVectorStore  # updated Pinecone wrapper
from pinecone import Pinecone, ServerlessSpec

# Create Pinecone client
pc = Pinecone(api_key=userdata.get('PINECONE'))  # or os.environ["PINECONE_API_KEY"]
index = pc.Index("q-a-index")

# Initialize embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# LangChain-compatible vector store
docsearch = PineconeVectorStore(index, embedding_model, text_key="content")


In [58]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,  # your Hugging Face LLM
    retriever=docsearch.as_retriever(search_kwargs={"k": 3}),
    return_source_documents=True
)

result = qa_chain({"query": user_query})
print(f"\n🧠 Answer:\n{result['result']}")


<ipython-input-58-4ff5be5cd3b1>:9: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  result = qa_chain({"query": user_query})
Token indices sequence length is longer than the specified maximum sequence length for this model (774 > 512). Running this sequence through the model will result in indexing errors



🧠 Answer:
Programming: Python, SQL Tools: Power BI, Tableau, Looker Studio, Git, Jupyter, Excel & Microsoft Tools, Streamlit, Fast API, AWS Libraries: Scikit-learn, NumPy, Pandas, Matplotlib, TensorFlow, OpenCV


# Recursive User query

In [60]:
while True:
    # Accept user query
    user_query = input("\n🔍 Enter your question (or type 'Q' to quit): ")

    # Check for quit condition
    if user_query.strip().lower() == 'q':
        print("👋 Exiting the Q&A system. Goodbye!")
        break

    # Embed the query using the same embedding model
    query_embedding = embedding_model.embed_query(user_query)

    # Retrieve and answer using QA chain
    result = qa_chain({"query": user_query})

    # Print response
    print(f"\n✅ Query: {user_query}")
    print(f"🧠 Answer: {result['result']}")



🔍 Enter your question (or type 'Q' to quit): What is the name of person in resume?

✅ Query: What is the name of person in resume?
🧠 Answer: Diksha Bisht

🔍 Enter your question (or type 'Q' to quit): What is her recent experience?

✅ Query: What is her recent experience?
🧠 Answer: Led a cross-functional team of 10 (developers, marketers, SEO experts) to deliver data-driven solutions for US-based clients across e-commerce, consulting, and tech domains.

🔍 Enter your question (or type 'Q' to quit): What was the nem of company and her role?

✅ Query: What was the nem of company and her role?
🧠 Answer: Embtel Solutions Pvt Ltd

🔍 Enter your question (or type 'Q' to quit): Q
👋 Exiting the Q&A system. Goodbye!
